In [ ]:
#initialization
from pyspark.sql import SparkSession
import json
import pandas as pd
import re
spark = SparkSession \
    .builder \
    .master("local[4]")\
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()\
    
    






In [ ]:
#load the dictionary
def load_dict():
    df = pd.read_excel("Dic_2014.xlsx",  parse_cols='A,E')
    df['Word'] = df['Word'].astype(str)
    df['Score'] = df['Score'].astype(int)
    _dic = df.set_index('Word').T.to_dict('list')
    return _dic


In [ ]:
#load data and change to rdd
def load_data(i):
    path ="12/"+i +"/*/*.json.bz2"
    twtr = spark.read.json(path)
    twtr = twtr.filter(twtr['lang'] == 'en')
    twtr_rdd = twtr.rdd
    return twtr_rdd


In [ ]:
#evalue te clean tweeter
def word_eva(clean_text,scores):
   
    scores['M'] = scores['M'] + 1
    list_words = clean_text.split(" ")
    #check dictionary
    for word in list_words:
        if word.upper() in senti_dic and senti_dic[word.upper()][0] >0:
            scores['P'] = scores['P'] +1
            return 0
        if word.upper() in senti_dic and senti_dic[word.upper()][0] <0:
            scores['N'] = scores['N'] + 1
            return 0
    return 0
    

In [ ]:
#map function
def map_dict(x):
    #dictionary to records scores
    score_dic = {'AMZN':{'M': 0,'P':0,'N':0},'IBM':{'M': 0,'P':0,'N':0},\
                 'TSLA':{'M': 0,'P':0,'N':0},"MSFT":{'M': 0,'P':0,'N':0},\
                 'VZ':{'M': 0,'P':0,'N':0}}
    in_str = str(x)
    
    clean_text = str(' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(text=u)",\
                                     " ", x['text']).split()))
    
  
    for i in ["AMZN","IBM", "TSLA","VZ","MSFT"]:
        if i in clean_text:
           
           
            word_eva(clean_text,score_dic[i])
           
            
            
    return score_dic

In [ ]:
#summary the scores
def reduce_dict(x,y):
    new_dict = x
    for i in x:
        for j in x[i]:
            new_dict[i][j] = x[i][j]+y[i][j]
    return new_dict
    

In [ ]:
################main################################
date_list = ["01","02","05","06","07","08","09",\
                 "12","13","14","15","16","19","20","21","22","23",\
            "27","28","29"]
#loop through files and record scores
score_list = {}
for i in date_list:
    date = "12-" +i
    senti_dic = load_dict()
    twtr_rdd = load_data(i)
    final_score = twtr_rdd.map(map_dict).reduce(reduce_dict)
    score_list[date] = final_score
 

In [ ]:
############################regression###########################################
import statsmodels.formula.api as sm
import pandas_datareader.data as web
import numpy as np
import datetime

In [ ]:
def get_params(tick):
    #get stock data
    start = datetime.datetime(2016, 12, 1)
    end = datetime.datetime(2016, 12, 31)
    f = web.DataReader("F", 'yahoo', start, end)
    f = f['Adj Close'].values
    #calculate return
    return_ = (np.array(f[1:])-np.array(f[0:-1]))/np.array(f[1:])
   
    # get the parameters
    df = pd.DataFrame()
    for i in score_list:
    
        seres = pd.DataFrame(score_list[i][tick] , [i,])
        df = df.append(seres)
    df=df.sort()
    df['S'] = return_

    #regression and get parameters
    result = sm.ols(formula="S ~ M + P", data=df).fit()

    return result.params

In [ ]:
#print out parameters
for i in ["AMZN","IBM", "TSLA","VZ","MSFT"]:
    print "the intercept, betas for ", i ," is \n ", get_params(i)